In [3]:
import numpy as np


class DataProcessor:
   # YOUR CODE HERE
    def __init__(self, data):
        self.data = data
        self.processed_data_ = None

    def process(self):
        self.processed_data_ = self.data - np.mean(self.data)

        return self
    
    def save_to_file(self, path: str):
        if self.processed_data_ is not None:
            with open(path, 'w') as f:
                for value in self.processed_data_:
                    f.write(str(value) + '\n')
        else:
            pass

# Example of usage
processor = DataProcessor(data=[1, 2, 3, 4, 5])
processor.process()
processor.save_to_file("processed_data.txt")

In [56]:
import os
from typing import Any
from typing import Tuple

import fire
import mlflow
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay

In [57]:
train_dataset = pd.read_csv('JUNIOR/ANTI_FRAUD_ONE/train.csv')
test_dataset = pd.read_csv('JUNIOR/ANTI_FRAUD_ONE/test.csv')

In [58]:
train_dataset.columns[:-1]

Index(['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11',
       'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21',
       'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30'],
      dtype='object')

In [5]:
train_dataset

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f22,f23,f24,f25,f26,f27,f28,f29,f30,target
0,-0.648624,0.568146,-0.459696,-0.916526,0.309660,-0.089906,-1.933051,-0.794830,0.542742,-1.583284,...,-0.374945,0.085396,-1.719799,0.646056,0.639181,-1.124101,0.777622,-2.047112,0.462838,0
1,0.164271,-0.272246,-1.096079,0.051618,-1.371842,1.396283,0.477539,0.527565,1.184486,0.551609,...,0.559933,0.715029,-0.701798,-0.941512,0.500928,-0.442733,-1.891319,-1.195447,-2.309315,0
2,-0.098563,0.202480,-1.200184,-1.442947,-1.389661,1.202339,-0.937037,-0.568139,1.310818,-0.952725,...,1.363509,0.764020,-1.076126,-0.156797,0.195455,-1.865436,-0.034823,-0.846974,0.283404,0
3,0.174284,0.407829,0.252578,1.369330,0.931167,-1.509193,1.162000,0.482860,0.315771,-0.749645,...,-0.117101,-1.307191,-0.384428,-0.017653,0.875763,-1.595071,0.975069,1.641830,-0.463939,0
4,-0.068953,0.600225,-0.039924,-0.197481,0.982254,-1.990336,1.402038,1.121289,-0.145707,0.454142,...,-1.039274,1.155712,0.762263,-0.043069,1.539298,-1.446338,1.600582,0.686237,0.194994,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1.965264,-1.285878,0.011498,1.653854,-0.654752,-1.632519,-0.513501,-0.508464,-1.243950,0.785503,...,-0.084012,-0.712171,-0.428628,0.136209,-0.144074,-0.663214,1.564126,-0.144282,-1.580001,0
19996,0.183041,0.449805,-1.440062,1.608024,0.061530,-0.427495,-1.578078,-1.688859,-1.205158,0.533792,...,-1.499487,-1.245609,0.232994,0.054031,-1.563273,-0.078073,0.612711,0.353185,0.673504,0
19997,-0.873557,-1.517115,0.717580,-0.809643,-1.054487,0.130939,0.375745,1.257669,-0.441016,0.436340,...,1.409140,0.635411,-1.377554,-0.470008,-1.669378,-1.314982,0.657474,0.037030,0.933878,0
19998,0.594632,1.989945,0.599970,1.038824,-1.573192,-0.179594,-1.213651,0.015227,1.426946,-1.291776,...,-2.103722,1.684042,1.329090,0.451009,0.979713,0.366301,0.863927,1.262408,-1.446170,0


In [6]:
test_dataset

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f22,f23,f24,f25,f26,f27,f28,f29,f30,target
0,-0.510512,-0.039430,0.817122,0.352355,-0.013309,-2.520200,-0.284907,0.156632,-1.697441,-2.540327,...,-0.882757,0.259968,-0.136287,0.674244,0.857626,-2.063846,0.327879,1.597144,-1.131871,0
1,0.107455,0.498242,1.684256,2.671831,-0.855473,0.814521,0.326930,0.596258,-0.548155,-0.869566,...,1.363089,-1.511199,0.181909,1.435273,-2.133867,-0.421446,-0.784693,0.145990,1.795862,0
2,-1.006121,0.224816,2.194780,0.153511,-1.529158,-0.483988,-1.724445,0.341533,0.354902,0.315454,...,-1.188751,-0.098770,-0.371024,-0.571466,0.249575,-0.257411,-1.158815,-0.094655,-0.088578,0
3,-0.924552,-1.381113,0.000166,-0.157032,0.381237,-2.147537,-1.040419,0.313838,-1.106434,1.073257,...,-1.288232,-0.545148,-1.695623,1.795284,1.734903,0.582160,-1.425357,-0.441730,0.813966,0
4,-0.406676,0.994885,1.535000,0.402133,0.994357,0.707093,0.478273,1.459609,0.817053,1.550470,...,0.737288,0.239144,0.364826,-0.299907,0.299424,-0.257399,-2.173846,1.153982,-0.120138,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.232556,-0.288375,-1.556947,0.322427,-0.320723,-0.208194,0.387382,0.863797,1.961913,-1.019652,...,-0.874472,0.484729,-1.045199,0.780685,-1.033691,0.609829,0.471403,-0.916557,0.892390,0
4996,-0.638300,0.771187,0.159978,-1.322216,-1.230197,0.301756,0.230986,-0.050581,0.871665,-0.411812,...,0.590942,0.581378,-0.929584,-1.333392,0.268427,0.719056,0.143759,0.899589,-0.602708,0
4997,0.928918,-1.031608,1.346215,0.002284,1.136789,2.093805,-1.495923,1.798096,1.300671,-0.770233,...,-0.887485,0.860002,0.415875,0.089295,0.120866,-0.195297,-0.073853,0.006267,0.059427,0
4998,0.958562,0.807185,1.542205,1.003087,1.693102,-0.821912,0.237149,0.433359,-0.479875,1.216178,...,0.632297,-1.614305,1.103043,-0.108668,1.943074,-0.257334,-0.022034,-0.998560,0.258756,0


In [7]:
model = IsolationForest(n_estimators=100)
model.fit(
    X=train_dataset.drop('target', axis=1)
)

IsolationForest()

In [8]:
test_targets = test_dataset['target']
pred_scores = -model.score_samples(test_dataset.drop('target', axis=1))

In [9]:
test_targets

0       0
1       0
2       0
3       0
4       0
       ..
4995    0
4996    0
4997    0
4998    0
4999    0
Name: target, Length: 5000, dtype: int64

In [10]:
vectorizer = np.vectorize(lambda x: 0 if x < 0.5 else 1)

In [13]:
test_targets

0       0
1       0
2       0
3       0
4       0
       ..
4995    0
4996    0
4997    0
4998    0
4999    0
Name: target, Length: 5000, dtype: int64

In [12]:
vectorizer(pred_scores)

array([0, 0, 0, ..., 0, 0, 0])

In [54]:
def recall_at_specificity(
    true_labels: np.ndarray,
    pred_scores: np.ndarray,
    min_specificity: float = 0.95,
) -> float:
    """Compute recall at specificity

    Args:
        true_labels (np.ndarray): True labels
        pred_scores (np.ndarray): Target scores
        min_specificity (float, optional): Min specificity for recall. Defaults to 0.95.

    Returns:
        float: Metric value
    """
    fpr, tpr, _ = roc_curve(
        y_true=true_labels,
        y_score=pred_scores
    )
    specificity = 1 - fpr 
    metric = max(tpr[specificity >= min_specificity])

    return metric

In [55]:
recall_at_specificity(
    true_labels=test_targets,
    pred_scores=pred_scores
)

0.9

In [28]:
def recall_at_precision(
    true_labels: np.ndarray,
    pred_scores: np.ndarray,
    min_precision: float = 0.95,
) -> float:
    """Compute recall at precision

    Args:
        true_labels (np.ndarray): True labels
        pred_scores (np.ndarray): Target scores
        min_precision (float, optional): Min precision for recall. Defaults to 0.95.

    Returns:
        float: Metric value
    """

    precision, recall, _ = precision_recall_curve(true_labels, pred_scores)
    print(precision, recall)
    print(recall[precision >= min_precision])
    metric = max(recall[precision >= min_precision])
    
    return metric

In [29]:
recall_at_precision(
    true_labels=test_targets,
    pred_scores=pred_scores
)

[0.01     0.010002 0.010004 ... 1.       1.       1.      ] [1.   1.   1.   ... 0.04 0.02 0.  ]
[0.28 0.26 0.24 0.22 0.2  0.18 0.16 0.14 0.12 0.1  0.08 0.06 0.04 0.02
 0.  ]


0.28